# convolstm run time

In [ ]:
######### mme4bat /   convlstim.py file ##################

In [2]:
# -*- coding: utf-8 -*-
"""ConvLSTM_running.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1TT_ptKSRw7tn9F5_mVK0kF9JjeLq4u0z
"""

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
from datetime import date,timedelta

import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

import time
import datetime

import numpy as np
import pandas as pd
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import RMSprop
from keras.callbacks import CSVLogger, EarlyStopping

In [10]:

path= '/data/notebook_files/DataOut/processed_data/bus_running_times_feature_added_all.csv'
data = pd.read_csv(path)

data = data.drop(data[data['run_time_in_seconds'] > 2000].index )
data = data.loc[(data['time_of_day']>= 6) & (data['time_of_day']<19)]
data = data[data['direction']==1]

In [11]:
print(data.columns)
print(data.head())

Index(['trip_id', 'deviceid', 'direction', 'segment', 'date', 'start_time',
       'end_time', 'run_time', 'run_time_in_seconds', 'length', 'day_of_week',
       'time_of_day', 'Sunday/holiday', 'saturday', 'weekday/end', 'week_no',
       'rt(w-1)', 'rt(w-2)', 'rt(w-3)', 'rt(t-1)', 'rt(t-2)', 'rt(n-1)',
       'rt(n-2)', 'rt(n-3)', 'hour_of_day', 'day', 'month', 'temp', 'precip',
       'windspeed', 'conditions', 'dt(n-1)'],
      dtype='object')
   trip_id  deviceid  direction  segment        date start_time  end_time  \
0      1.0     262.0        1.0      1.0  2021-10-01   06:39:49  06:40:58   
1      1.0     262.0        1.0      2.0  2021-10-01   06:42:12  06:45:42   
2      1.0     262.0        1.0      3.0  2021-10-01   06:45:42  06:53:58   
3      1.0     262.0        1.0      4.0  2021-10-01   06:54:04  06:57:19   
4      1.0     262.0        1.0      5.0  2021-10-01   06:57:19  06:58:56   

          run_time  run_time_in_seconds  length  ...  rt(n-2)  rt(n-3)  \
0  0 days 0

In [4]:

"""PrepData

"""

data['DateTime'] = pd.to_datetime(data['date'] + ' ' + data['start_time'])

ref_freq = '15min'
ix = pd.DatetimeIndex(pd.to_datetime(data['DateTime'])).floor(ref_freq)

data["DateTimeRef"] = ix

data['DayOfWeek'] = pd.to_datetime(data['date']).dt.weekday
data["DowTimeRef"] = pd.to_datetime((data['DayOfWeek'].values ) * 24 * 60 * 60 + ix.hour * 60 * 60 + ix.minute * 60, unit = 's')

data

,trip_id,deviceid,direction,segment,date,start_time,end_time,run_time,run_time_in_seconds,length,...,month,temp,precip,windspeed,conditions,dt(n-1),DateTime,DateTimeRef,DayOfWeek,DowTimeRef
0,1.0,262.0,1.0,1.0,2021-10-01,06:39:49,06:40:58,0 days 00:01:09,69.0,0.6261,...,10.0,20.0,0.0,6.1,Partially cloudy,0.0,2021-10-01 06:39:49,2021-10-01 06:30:00,4,1970-01-05 06:30:00
1,1.0,262.0,1.0,2.0,2021-10-01,06:42:12,06:45:42,0 days 00:03:30,210.0,1.2808,...,10.0,20.0,0.0,6.1,Partially cloudy,74.0,2021-10-01 06:42:12,2021-10-01 06:30:00,4,1970-01-05 06:30:00
2,1.0,262.0,1.0,3.0,2021-10-01,06:45:42,06:53:58,0 days 00:08:16,496.0,2.1125,...,10.0,20.0,0.0,6.1,Partially cloudy,0.0,2021-10-01 06:45:42,2021-10-01 06:45:00,4,1970-01-05 06:45:00
3,1.0,262.0,1.0,4.0,2021-10-01,06:54:04,06:57:19,0 days 00:03:15,195.0,1.5513,...,10.0,20.0,0.0,6.1,Partially cloudy,6.0,2021-10-01 06:54:04,2021-10-01 06:45:00,4,1970-01-05 06:45:00
4,1.0,262.0,1.0,5.0,2021-10-01,06:57:19,06:58:56,0 days 00:01:37,97.0,0.8450,...,10.0,20.0,0.0,6.1,Partially cloudy,0.0,2021-10-01 06:57:19,2021-10-01 06:45:00,4,1970-01-05 06:45:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173720,25370.0,121.0,1.0,11.0,2022-11-01,18:47:53,18:50:08,0:02:15,135.0,1.1000,...,11.0,20.5,0.0,8.3,Overcast,15.0,2022-11-01 18:47:53,2022-11-01 18:45:00,1,1970-01-02 18:45:00
173721,25370.0,121.0,1.0,12.0,2022-11-01,18:50:15,18:52:39,0:02:24,144.0,1.3100,...,11.0,20.5,0.0,8.3,Overcast,7.0,2022-11-01 18:50:15,2022-11-01 18:45:00,1,1970-01-02 18:45:00
173722,25370.0,121.0,1.0,13.0,2022-11-01,18:52:50,18:55:01,0:02:11,131.0,1.1500,...,11.0,20.5,0.0,8.3,Overcast,11.0,2022-11-01 18:52:50,2022-11-01 18:45:00,1,1970-01-02 18:45:00
173723,25370.0,121.0,1.0,14.0,2022-11-01,18:55:14,18:56:55,0:01:41,101.0,0.8900,...,11.0,20.5,0.0,8.3,Overcast,13.0,2022-11-01 18:55:14,2022-11-01 18:45:00,1,1970-01-02 18:45:00


In [5]:


def datetime_index_generator(start_date,end_date,start_time,end_time,freq):
  date_range = pd.date_range(start_date, end_date, freq='D')
  date_list = [i.strftime('%Y-%m-%d') for i in date_range]
  datetime_series = pd.Series()

  for date in date_list:
    begin_timestamp = pd.to_datetime(date + ' ' + start_time )
    end_timestamp = pd.to_datetime(date + ' ' + end_time )
    daily_time_range = pd.Series(index = pd.date_range(begin_timestamp,end_timestamp,freq = freq,closed='left'))

    datetime_series = pd.concat([datetime_series,daily_time_range ]).sort_index()

  return datetime_series

In [6]:

"""Fit_scale"""

def fit_scale(data, ref_freq = '15min'):
  means = { }
  scales = { }
  low = { }
  upr = { }

  grouping = data[data['run_time_in_seconds'].notnull()].groupby('segment', sort = False)

  for segment, data_link in grouping:
        # Fit outlier bounds using MAD
        median = data_link.groupby('DowTimeRef')['run_time_in_seconds'].median()
        error = pd.concat([data_link['DowTimeRef'], np.abs(data_link['run_time_in_seconds'] - median[data_link['DowTimeRef']].values)], axis = 1)
        mad = 1.4826 * error.groupby('DowTimeRef')['run_time_in_seconds'].median()
        
        _low = median - 3 * mad
        _upr = median + 3 * mad
        mask = (_low[data_link['DowTimeRef']].values < data_link['run_time_in_seconds']) & (data_link['run_time_in_seconds'] < _upr[data_link['DowTimeRef']].values)
        data_link_no = data_link[mask]
        
        _mean = data_link_no.groupby('DowTimeRef')["run_time_in_seconds"].mean()
        means[segment] = _mean
        low[segment] = _low
        upr[segment] = _upr
        scales[segment] = data_link_no['run_time_in_seconds'].std()

  #ix = pd.date_range('1970-01-01', '1970-01-08', freq = ref_freq, closed = 'left')
  index_series = datetime_index_generator('1970-01-01','1970-01-07','06:00:00','19:00:00',freq='15min')
  ix = index_series.index
  means_df = pd.DataFrame(data = means, index = ix).interpolate()
  low_df = pd.DataFrame(data = low, index = ix).interpolate()
  upr_df = pd.DataFrame(data = upr, index = ix).interpolate()

  means_df = means_df.fillna(method='pad').fillna(method='bfill')
  low_df = low_df.fillna(method='pad').fillna(method='bfill')
  upr_df = upr_df.fillna(method='pad').fillna(method='bfill')

  return means_df, scales, low_df, upr_df

In [7]:

"""Time series index generator"""

s1 = datetime_index_generator('2021-10-01','2022-02-28','06:00:00','19:00:00',freq='15min')
s2 = datetime_index_generator('2022-07-01','2022-11-01','06:00:00','19:00:00',freq='15min')

s = pd.concat([s1,s2]).sort_index()

datetime_index = s.index

datetime_index

<ipython-input-5-0b257dfc7884>:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  datetime_series = pd.Series()
<ipython-input-5-0b257dfc7884>:9: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  daily_time_range = pd.Series(index = pd.date_range(begin_timestamp,end_timestamp,freq = freq,closed='left'))
<ipython-input-5-0b257dfc7884>:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  daily_time_range = pd.Series(index = pd.date_range(begin_timestamp,end_timestamp,freq = freq,closed='left'))
<ipython-input-5-0b257dfc7884>:9: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  daily_time_range = pd.Series(index = pd.date_range(begin_timestamp,end_timestamp,freq = freq,closed='left'))
<ipython-input-5-0b257dfc7884>:9:

DatetimeIndex(['2021-10-01 06:00:00', '2021-10-01 06:15:00',
               '2021-10-01 06:30:00', '2021-10-01 06:45:00',
               '2021-10-01 07:00:00', '2021-10-01 07:15:00',
               '2021-10-01 07:30:00', '2021-10-01 07:45:00',
               '2021-10-01 08:00:00', '2021-10-01 08:15:00',
               ...
               '2022-11-01 16:30:00', '2022-11-01 16:45:00',
               '2022-11-01 17:00:00', '2022-11-01 17:15:00',
               '2022-11-01 17:30:00', '2022-11-01 17:45:00',
               '2022-11-01 18:00:00', '2022-11-01 18:15:00',
               '2022-11-01 18:30:00', '2022-11-01 18:45:00'],
              dtype='datetime64[ns]', length=14300, freq=None)

In [8]:

"""Outliers"""

def remove_outliers(data, low, upr): 
    _low = low.lookup(data['DowTimeRef'], data['segment'])
    _upr = upr.lookup(data['DowTimeRef'], data['segment'])
    mask = ((_low < data['run_time_in_seconds']) & (data['run_time_in_seconds'] < _upr))
    data = data.loc[mask].copy()
    return data, (~mask).sum()

In [9]:

"""Transform"""

def transform(data, means_df, scales,datetime_index, freq = '15min'):
  tss = { }
  ws = { }
  removed_mean = { }
  removed_scale = { }
  ks = []
  
  for k, v in data.groupby('segment', sort = False):
        # Link Data Time Indexed
        link_time_ix = pd.DatetimeIndex(pd.to_datetime(v['DateTime']))    
        link_time_ixd = v.set_index(link_time_ix)
        
        # Link Reference Data Index
        ix_ref = link_time_ixd['DowTimeRef']  

        link_travel_time_k = link_time_ixd['run_time_in_seconds'].resample(freq).mean()
        link_travel_time_k = link_travel_time_k[link_travel_time_k.index.isin(datetime_index)]
        
        removed_mean[k] = pd.Series(data = means_df.loc[ix_ref, k].values, index = link_time_ix).resample(freq).mean()
        removed_mean[k] = removed_mean[k][removed_mean[k].index.isin(datetime_index)]

        removed_scale[k] = pd.Series(data = np.repeat(scales[k], link_travel_time_k.shape[0]), index = link_travel_time_k.index)
        tss[k] = (link_travel_time_k - removed_mean[k].values) / removed_scale[k].values
        
        ws[k] = link_time_ixd['run_time_in_seconds'].resample(freq).count()
        ws[k] = ws[k][ws[k].index.isin(datetime_index)]

        ks.append(k)

  ts = pd.DataFrame(data = tss).fillna(method='pad').fillna(0) # Link Travel Time Time Series
  df_removed_mean = pd.DataFrame(data = removed_mean, index = ts.index).fillna(method='pad').fillna(method='bfill') # Removed Mean from Link Travel Time
  df_removed_scale = pd.DataFrame(data = removed_scale, index = ts.index).fillna(method='pad').fillna(method='bfill')
  w = pd.DataFrame(data = ws).fillna(0) # Link Travel Time Weights, e.g. number of measurements

  return (ts.index, ts.values, df_removed_mean.values, df_removed_scale.values, w.values, ks)

In [10]:

def roll(ix, ts, removed_mean, removed_scale, w, lags, preds):
    X = np.stack([np.roll(ts, i, axis = 0) for i in range(lags, 0, -1)], axis = 1)[lags:-preds,]
    Y = np.stack([np.roll(ts, -i, axis = 0) for i in range(0, preds, 1)], axis = 1)[lags:-preds,]
    Y_ix = ix[lags:-preds]
    Y_mean = np.stack([np.roll(removed_mean, -i, axis = 0) for i in range(0, preds, 1)], axis = 1)[lags:-preds,]
    Y_scale = np.stack([np.roll(removed_scale, -i, axis = 0) for i in range(0, preds, 1)], axis = 1)[lags:-preds,]
    w_y = np.stack([np.roll(w, -i, axis = 0) for i in range(0, preds, 1)], axis = 1)[lags:-preds,]

    return X, Y, Y_ix, Y_mean, Y_scale, w_y

In [11]:

"""Model Definition"""

def build_model(input_timesteps, output_timesteps, num_links):
    model = Sequential()
    model.add(BatchNormalization(name = 'batch_norm_0', input_shape = (input_timesteps, num_links, 1, 1)))
    model.add(ConvLSTM2D(name ='conv_lstm_1',
                         filters = 64, kernel_size = (10, 1),                       
                         padding = 'same', 
                         return_sequences = True))
    
    model.add(Dropout(0.2, name = 'dropout_1'))
    model.add(BatchNormalization(name = 'batch_norm_1'))

    model.add(ConvLSTM2D(name ='conv_lstm_2',
                         filters = 64, kernel_size = (5, 1), 
                         padding='same',
                         return_sequences = False))
    
    model.add(Dropout(0.1, name = 'dropout_2'))
    model.add(BatchNormalization(name = 'batch_norm_2'))
    
    model.add(Flatten())
    model.add(RepeatVector(output_timesteps))
    model.add(Reshape((output_timesteps, num_links, 1, 64)))
    
    model.add(ConvLSTM2D(name ='conv_lstm_3',
                         filters = 64, kernel_size = (10, 1), 
                         padding='same',
                         return_sequences = True))
    
    model.add(Dropout(0.1, name = 'dropout_3'))
    model.add(BatchNormalization(name = 'batch_norm_3'))
    
    model.add(ConvLSTM2D(name ='conv_lstm_4',
                         filters = 64, kernel_size = (5, 1), 
                         padding='same',
                         return_sequences = True))
    
    model.add(TimeDistributed(Dense(units=1, name = 'dense_1', activation = 'relu')))
    #model.add(TimeDistributed(Dense(units=1, name = 'dense_1', activation = 'relu')))
    #model.add(Dense(units=1, name = 'dense_2'))

    optimizer = RMSprop(lr=0.0001, rho=0.9, epsilon=1e-08, decay=0.9)
    model.compile(loss = "mse", optimizer = optimizer)
    return model


In [12]:

def info(msg):
    print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S') + " " + msg)

In [13]:

"""Load Data"""



lags = 4 * 13
preds = 3

n = len(data)

In [15]:

"""Train and Test"""

data_train = data[data['week_no'] < 36]
data_test = data[36 <= data['week_no']]
n_train = len(data_train)
n_test = len(data_test)
info('- Train size : {:>8} ({:.2f}%) '.format(n_train, 100. * n_train / n))
info('- Test size  : {:>8} ({:.2f}%) '.format(n_test, 100. * n_test / n))

(means, scales, low, upr) = fit_scale(data_train)

data_train_no , no_removed =  remove_outliers(data_train,low,upr)

# no_removed

data_test_no , no_removed =  remove_outliers(data_test,low,upr)
# no_removed

#data_train_no = data_train
#data_test_no = data_test

2023-09-20 13:34:24 - Train size :    78957 (80.71%) 
2023-09-20 13:34:24 - Test size  :    18876 (19.29%) 


<ipython-input-5-0b257dfc7884>:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  datetime_series = pd.Series()
<ipython-input-5-0b257dfc7884>:9: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  daily_time_range = pd.Series(index = pd.date_range(begin_timestamp,end_timestamp,freq = freq,closed='left'))
<ipython-input-5-0b257dfc7884>:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  daily_time_range = pd.Series(index = pd.date_range(begin_timestamp,end_timestamp,freq = freq,closed='left'))
<ipython-input-5-0b257dfc7884>:9: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  daily_time_range = pd.Series(index = pd.date_range(begin_timestamp,end_timestamp,freq = freq,closed='left'))
<ipython-input-5-0b257dfc7884>:9:

In [17]:

ix_train, ts_train, rm_mean_train, rm_scale_train, w_train, lns_train = transform(data_train_no, means, scales,datetime_index)
ix_test, ts_test, rm_mean_test, rm_scale_test, w_test, lns_test = transform(data_test_no, means, scales,datetime_index)

#Create rolling window tensor
X_train, Y_train, Y_ix_train, Y_rm_mean_train, Y_scale_train, Y_w_train = roll(ix_train, ts_train, rm_mean_train, rm_scale_train, w_train, lags, preds)
X_test, Y_test, Y_ix_test, Y_rm_mean_test, Y_scale_test, Y_w_test = roll(ix_test, ts_test, rm_mean_test, rm_scale_test, w_test, lags, preds)

X_train = X_train[:,:,:,np.newaxis,np.newaxis]
Y_train = Y_train[:,:,:,np.newaxis,np.newaxis]
X_test = X_test[:,:,:,np.newaxis,np.newaxis]
Y_test = Y_test[:,:,:,np.newaxis,np.newaxis]

info('- X_train shape : {}    X_test shape : {}'.format(X_train.shape, X_test.shape))
info('- Y_train shape : {}    Y_test shape : {}'.format(Y_train.shape, Y_test.shape))

2023-09-20 13:38:42 - X_train shape : (11955, 52, 15, 1, 1)    X_test shape : (2232, 52, 15, 1, 1)
2023-09-20 13:38:42 - Y_train shape : (11955, 3, 15, 1, 1)    Y_test shape : (2232, 3, 15, 1, 1)


In [18]:

global_start_time = time.time()
early_stopping = EarlyStopping(monitor='val_loss', patience = 3)

model = build_model(lags, preds, len(lns_train))

# Train
history = model.fit(X_train, Y_train,
                    batch_size = 32, epochs = 1,
                    shuffle = False, validation_data = (X_test, Y_test),
                    verbose = 2, callbacks = [early_stopping])
model.save('models/ConvLSTM_3x15min_10x64-5x64-10x64-5x64_Comparison.h5') 

374/374 - 1457s - loss: 0.8594 - val_loss: 0.8537 - 1457s/epoch - 4s/step


/opt/python/envs/default/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [19]:

Y_true = Y_test.squeeze() * Y_scale_test + Y_rm_mean_test
Y_naive = Y_rm_mean_test
Y_pred = model.predict(X_test).squeeze() * Y_scale_test + Y_rm_mean_test

Y_true_total = np.sum(Y_true * Y_w_test, axis = 2).squeeze()
Y_naive_total = np.sum(Y_naive * Y_w_test, axis = 2).squeeze()
Y_pred_total = np.sum(Y_pred * Y_w_test, axis = 2).squeeze()
  

70/70 [==============================] - 105s 1s/step


In [20]:
  
for t in range(preds):
    mask = Y_true_total[:,t] > 0
    Y_true_total_t = Y_true_total[mask, t] / 60
    Y_naive_total_t = Y_naive_total[mask, t] / 60
    Y_pred_total_t = Y_pred_total[mask, t] / 60  

    error_naive_total_t = (Y_naive_total_t - Y_true_total_t)
    error_lstm_total_t = (Y_pred_total_t - Y_true_total_t)

    mae_lstm = np.mean(np.abs(error_lstm_total_t))
    rmse_lstm = np.sqrt(np.mean((error_lstm_total_t)**2))
    mape_lstm = np.mean(np.abs(error_lstm_total_t) / Y_true_total_t) * 100

    info("- t + %d - ConvLSTM - MAE: %5.2f - RMSE: %5.2f - MAPE: %5.2f" % (t + 1, mae_lstm, rmse_lstm, mape_lstm))

model.summary()

2023-09-20 14:05:38 - t + 1 - ConvLSTM - MAE:  1.79 - RMSE:  2.45 - MAPE:  8.14
2023-09-20 14:05:38 - t + 2 - ConvLSTM - MAE:  1.79 - RMSE:  2.45 - MAPE:  8.15
2023-09-20 14:05:38 - t + 3 - ConvLSTM - MAE:  1.80 - RMSE:  2.46 - MAPE:  8.17
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_norm_0 (BatchNormaliz  (None, 52, 15, 1, 1)     4         
 ation)                                                          
                                                                 
 conv_lstm_1 (ConvLSTM2D)    (None, 52, 15, 1, 64)     166656    
                                                                 
 dropout_1 (Dropout)         (None, 52, 15, 1, 64)     0         
                                                                 
 batch_norm_1 (BatchNormaliz  (None, 52, 15, 1, 64)    256       
 ation)                                                          
              

In [21]:

# !pip install neuralplot


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


ValueError: ValueError: shape mismatch: objects cannot be broadcast to a single shape.  Mismatch is between arg 0 with shape (780,) and arg 1 with shape (225,).

In [ ]:


output = pd.DataFrame(data = Y_pred[:,0,:], index = Y_ix_test, columns = lns_test)

output['Datetime']=output.index

output


In [ ]:
pred_run_convolstm = output

In [ ]:


def download_csv(data,filename):
  filename= filename + '.csv'
  data.to_csv(filename, encoding = 'utf-8-sig',index= False)
  
download_csv(pred_run_convolstm,'/predicted_values/predicted_running_times_convolstm')


In [ ]:

# """Out of Fold Training"""



# global_start_time = time.time()
# early_stopping = EarlyStopping(monitor='val_loss', patience = 3)

# bootstrap_size_pct = 0.80
# test_window_pct = 0.05
# max_iter = 4

# lags = 4 * 8
# preds = 3

# hist = []
# for i in range(max_iter):

#     info("Current window: " + str(i))
    
#     # Devide into test and train
#     data_train = data[:int((bootstrap_size_pct + i * test_window_pct) * n)]
#     data_test = data[int((bootstrap_size_pct + i * test_window_pct) * n):int((bootstrap_size_pct + (i + 1) * test_window_pct) * n)]
#     n_train = len(data_train)
#     n_test = len(data_test)
#     info('- Train size : {:>8} ({:.2f}%) '.format(n_train, 100. * n_train / n))
#     info('- Test size  : {:>8} ({:.2f}%) '.format(n_test, 100. * n_test / n))
    
#     # Mean center and scale
#     (means, scales, low, upr) = fit_scale(data_train)
#     assert means.shape == (4 * 24 * 7, 32)
#     assert len(scales) == 32
#     assert low.shape == (4 * 24 * 7, 32)
#     assert upr.shape == (4 * 24 * 7, 32)
    
#     data_train_no = data_train
#     data_test_no = data_test
#     #data_train_no, n_outliers = remove_outliers(data_train, low, upr)
#     #info('- Removed {0} outliers ({1:.2f}%) from train'.format(n_outliers, 100.0 * n_outliers / len(data_train)))
#     #data_test_no, n_outliers = remove_outliers(data_test, low, upr)
#     #info('- Removed {0} outliers ({1:.2f}%) from test'.format(n_outliers, 100.0 * n_outliers / len(data_test)))
    
#     ix_train, ts_train, rm_mean_train, rm_scale_train, w_train, lns_train = transform(data_train_no, means, scales)
#     ix_test, ts_test, rm_mean_test, rm_scale_test, w_test, lns_test = transform(data_test_no, means, scales)

# X_train, Y_train, Y_ix_train, Y_rm_mean_train, Y_scale_train, Y_w_train = roll(ix_train, ts_train, rm_mean_train, rm_scale_train, w_train, lags, preds)
# X_test, Y_test, Y_ix_test, Y_rm_mean_test, Y_scale_test, Y_w_test = roll(ix_test, ts_test, rm_mean_test, rm_scale_test, w_test, lags, preds)
# X_train = X_train[:,:,:,np.newaxis,np.newaxis]
# Y_train = Y_train[:,:,:,np.newaxis,np.newaxis]
# X_test = X_test[:,:,:,np.newaxis,np.newaxis]
# Y_test = Y_test[:,:,:,np.newaxis,np.newaxis]
    
# info('- X_train shape : {:>20}    X_test shape : {:>20}'.format(X_train.shape, X_test.shape))
# info('- Y_train shape : {:>20}    Y_test shape : {:>20}'.format(Y_train.shape, Y_test.shape))
    
# model = build_model(lags, preds, len(lns_train))
    
#     # Train
# history = model.fit(X_train, Y_train,
#                         batch_size = 128, epochs = 30,
#                         shuffle = False, validation_data = (X_test, Y_test),
#                         verbose = 2) #, callbacks = [csv_logger, early_stopping])
# hist.append(history)
# model.save('models/ConvLSTM_3x15min_10x64-5x64-10x64-5x64_' + str(i) + '.h5') 

# Y_true = Y_test.squeeze() * Y_scale_test + Y_rm_mean_test
# Y_naive = Y_rm_mean_test
# Y_pred = model.predict(X_test).squeeze() * Y_scale_test + Y_rm_mean_test
        
# Y_true_total = np.sum(Y_true * Y_w_test, axis = 2).squeeze()
# Y_naive_total = np.sum(Y_naive * Y_w_test, axis = 2).squeeze()
# Y_pred_total = np.sum(Y_pred * Y_w_test, axis = 2).squeeze()


# convolstm bus stops

In [1]:
######### mme4bat /   convlstim.py file ##################

In [12]:
# -*- coding: utf-8 -*-
"""ConvLSTM_running.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1TT_ptKSRw7tn9F5_mVK0kF9JjeLq4u0z
"""

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
from datetime import date,timedelta

import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

import time
import datetime

import numpy as np
import pandas as pd
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import RMSprop
from keras.callbacks import CSVLogger, EarlyStopping

In [13]:

path= '/data/notebook_files/DataOut/processed_data/bus_stop_times_feature_added_all.csv'
data = pd.read_csv(path)

In [14]:

data = data.drop(data[data['dwell_time_in_seconds'] > 2000].index )
data = data.loc[(data['time_of_day']>= 6) & (data['time_of_day']<19)]
data = data[data['direction']==1]

In [15]:
print(data.columns)
print(data.head())

Index(['trip_id', 'deviceid', 'direction', 'bus_stop', 'date', 'arrival_time',
       'departure_time', 'dwell_time', 'dwell_time_in_seconds_old',
       'day_of_week', 'time_of_day', 'Sunday/holiday', 'saturday',
       'weekday/end', 'week_no', 'dt(w-1)', 'dt(w-2)', 'dt(w-3)', 'dt(t-1)',
       'dt(t-2)', 'dt(n-1)', 'dt(n-2)', 'dt(n-3)', 'hour_of_day', 'day',
       'month', 'temp', 'precip', 'windspeed', 'conditions', 'rt(n-1)',
       'stop_type', 'dwell_time_in_seconds'],
      dtype='object')
   trip_id  deviceid  direction  bus_stop        date arrival_time  \
0        1       262          1       101  2021-10-01     06:40:58   
1        1       262          1       102  2021-10-01     06:45:42   
2        1       262          1       103  2021-10-01     06:53:58   
3        1       262          1       104  2021-10-01     06:57:19   
4        1       262          1       105  2021-10-01     06:58:56   

  departure_time dwell_time  dwell_time_in_seconds_old  day_of_week  ...  \

In [16]:

"""PrepData

"""

data['DateTime'] = pd.to_datetime(data['date'] + ' ' + data['arrival_time'])

ref_freq = '15min'
ix = pd.DatetimeIndex(pd.to_datetime(data['DateTime'])).floor(ref_freq)

data["DateTimeRef"] = ix

data['DayOfWeek'] = pd.to_datetime(data['date']).dt.weekday
data["DowTimeRef"] = pd.to_datetime((data['DayOfWeek'].values ) * 24 * 60 * 60 + ix.hour * 60 * 60 + ix.minute * 60, unit = 's')

data

,trip_id,deviceid,direction,bus_stop,date,arrival_time,departure_time,dwell_time,dwell_time_in_seconds_old,day_of_week,...,precip,windspeed,conditions,rt(n-1),stop_type,dwell_time_in_seconds,DateTime,DateTimeRef,DayOfWeek,DowTimeRef
0,1,262,1,101,2021-10-01,06:40:58,06:42:12,0:01:14,74.0,4,...,0.0,6.1,Partially cloudy,69.0,pro,74.0,2021-10-01 06:40:58,2021-10-01 06:30:00,4,1970-01-05 06:30:00
1,1,262,1,102,2021-10-01,06:45:42,06:45:42,0:00:00,0.0,4,...,0.0,6.1,Partially cloudy,210.0,mod,0.0,2021-10-01 06:45:42,2021-10-01 06:45:00,4,1970-01-05 06:45:00
2,1,262,1,103,2021-10-01,06:53:58,06:54:04,0:00:06,6.0,4,...,0.0,6.1,Partially cloudy,496.0,mod,6.0,2021-10-01 06:53:58,2021-10-01 06:45:00,4,1970-01-05 06:45:00
3,1,262,1,104,2021-10-01,06:57:19,06:57:19,0:00:00,0.0,4,...,0.0,6.1,Partially cloudy,195.0,br,0.0,2021-10-01 06:57:19,2021-10-01 06:45:00,4,1970-01-05 06:45:00
4,1,262,1,105,2021-10-01,06:58:56,07:02:27,0:03:31,211.0,4,...,0.0,6.1,Partially cloudy,97.0,pro,211.0,2021-10-01 06:58:56,2021-10-01 06:45:00,4,1970-01-05 06:45:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91678,25370,121,1,110,2022-11-01,18:47:38,18:47:53,0:00:15,15.0,1,...,0.0,8.3,Overcast,134.0,mod,15.0,2022-11-01 18:47:38,2022-11-01 18:45:00,1,1970-01-02 18:45:00
91679,25370,121,1,111,2022-11-01,18:50:08,18:50:15,0:00:07,7.0,1,...,0.0,8.3,Overcast,135.0,br,7.0,2022-11-01 18:50:08,2022-11-01 18:45:00,1,1970-01-02 18:45:00
91680,25370,121,1,112,2022-11-01,18:52:39,18:52:50,0:00:11,11.0,1,...,0.0,8.3,Overcast,144.0,br,11.0,2022-11-01 18:52:39,2022-11-01 18:45:00,1,1970-01-02 18:45:00
91681,25370,121,1,113,2022-11-01,18:55:01,18:55:14,0:00:13,13.0,1,...,0.0,8.3,Overcast,131.0,mod,13.0,2022-11-01 18:55:01,2022-11-01 18:45:00,1,1970-01-02 18:45:00


In [17]:


def datetime_index_generator(start_date,end_date,start_time,end_time,freq):
  date_range = pd.date_range(start_date, end_date, freq='D')
  date_list = [i.strftime('%Y-%m-%d') for i in date_range]
  datetime_series = pd.Series()

  for date in date_list:
    begin_timestamp = pd.to_datetime(date + ' ' + start_time )
    end_timestamp = pd.to_datetime(date + ' ' + end_time )
    daily_time_range = pd.Series(index = pd.date_range(begin_timestamp,end_timestamp,freq = freq,closed='left'))

    datetime_series = pd.concat([datetime_series,daily_time_range ]).sort_index()

  return datetime_series

In [18]:

"""Fit_scale"""

def fit_scale(data, ref_freq = '15min'):
  means = { }
  scales = { }
  low = { }
  upr = { }

  grouping = data[data['dwell_time_in_seconds'].notnull()].groupby('bus_stop', sort = False)

  for segment, data_link in grouping:
        # Fit outlier bounds using MAD
        median = data_link.groupby('DowTimeRef')['dwell_time_in_seconds'].median()
        error = pd.concat([data_link['DowTimeRef'], np.abs(data_link['dwell_time_in_seconds'] - median[data_link['DowTimeRef']].values)], axis = 1)
        mad = 1.4826 * error.groupby('DowTimeRef')['dwell_time_in_seconds'].median()
        
        _low = median - 3 * mad
        _upr = median + 3 * mad
        mask = (_low[data_link['DowTimeRef']].values < data_link['dwell_time_in_seconds']) & (data_link['dwell_time_in_seconds'] < _upr[data_link['DowTimeRef']].values)
        data_link_no = data_link[mask]
        
        _mean = data_link_no.groupby('DowTimeRef')["dwell_time_in_seconds"].mean()
        means[segment] = _mean
        low[segment] = _low
        upr[segment] = _upr
        scales[segment] = data_link_no['dwell_time_in_seconds'].std()

  #ix = pd.date_range('1970-01-01', '1970-01-08', freq = ref_freq, closed = 'left')
  index_series = datetime_index_generator('1970-01-01','1970-01-07','06:00:00','19:00:00',freq='15min')
  ix = index_series.index
  means_df = pd.DataFrame(data = means, index = ix).interpolate()
  low_df = pd.DataFrame(data = low, index = ix).interpolate()
  upr_df = pd.DataFrame(data = upr, index = ix).interpolate()

  means_df = means_df.fillna(method='pad').fillna(method='bfill')
  low_df = low_df.fillna(method='pad').fillna(method='bfill')
  upr_df = upr_df.fillna(method='pad').fillna(method='bfill')

  return means_df, scales, low_df, upr_df

In [19]:

"""Time series index generator"""

s1 = datetime_index_generator('2021-10-01','2022-02-28','06:00:00','19:00:00',freq='15min')
s2 = datetime_index_generator('2022-07-01','2022-11-01','06:00:00','19:00:00',freq='15min')

s = pd.concat([s1,s2]).sort_index()

datetime_index = s.index

datetime_index

<ipython-input-17-0b257dfc7884>:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  datetime_series = pd.Series()
<ipython-input-17-0b257dfc7884>:9: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  daily_time_range = pd.Series(index = pd.date_range(begin_timestamp,end_timestamp,freq = freq,closed='left'))
<ipython-input-17-0b257dfc7884>:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  daily_time_range = pd.Series(index = pd.date_range(begin_timestamp,end_timestamp,freq = freq,closed='left'))
<ipython-input-17-0b257dfc7884>:9: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  daily_time_range = pd.Series(index = pd.date_range(begin_timestamp,end_timestamp,freq = freq,closed='left'))
<ipython-input-17-0b257dfc788

DatetimeIndex(['2021-10-01 06:00:00', '2021-10-01 06:15:00',
               '2021-10-01 06:30:00', '2021-10-01 06:45:00',
               '2021-10-01 07:00:00', '2021-10-01 07:15:00',
               '2021-10-01 07:30:00', '2021-10-01 07:45:00',
               '2021-10-01 08:00:00', '2021-10-01 08:15:00',
               ...
               '2022-11-01 16:30:00', '2022-11-01 16:45:00',
               '2022-11-01 17:00:00', '2022-11-01 17:15:00',
               '2022-11-01 17:30:00', '2022-11-01 17:45:00',
               '2022-11-01 18:00:00', '2022-11-01 18:15:00',
               '2022-11-01 18:30:00', '2022-11-01 18:45:00'],
              dtype='datetime64[ns]', length=14300, freq=None)

In [20]:

"""Outliers"""

def remove_outliers(data, low, upr): 
    _low = low.lookup(data['DowTimeRef'], data['bus_stop'])
    _upr = upr.lookup(data['DowTimeRef'], data['bus_stop'])
    mask = ((_low < data['dwell_time_in_seconds']) & (data['dwell_time_in_seconds'] < _upr))
    data = data.loc[mask].copy()
    return data, (~mask).sum()

In [21]:

"""Transform"""

def transform(data, means_df, scales,datetime_index, freq = '15min'):
  tss = { }
  ws = { }
  removed_mean = { }
  removed_scale = { }
  ks = []
  
  for k, v in data.groupby('bus_stop', sort = False):
        # Link Data Time Indexed
        link_time_ix = pd.DatetimeIndex(pd.to_datetime(v['DateTime']))    
        link_time_ixd = v.set_index(link_time_ix)
        
        # Link Reference Data Index
        ix_ref = link_time_ixd['DowTimeRef']  

        link_travel_time_k = link_time_ixd['dwell_time_in_seconds'].resample(freq).mean()
        link_travel_time_k = link_travel_time_k[link_travel_time_k.index.isin(datetime_index)]
        
        removed_mean[k] = pd.Series(data = means_df.loc[ix_ref, k].values, index = link_time_ix).resample(freq).mean()
        removed_mean[k] = removed_mean[k][removed_mean[k].index.isin(datetime_index)]

        removed_scale[k] = pd.Series(data = np.repeat(scales[k], link_travel_time_k.shape[0]), index = link_travel_time_k.index)
        tss[k] = (link_travel_time_k - removed_mean[k].values) / removed_scale[k].values
        
        ws[k] = link_time_ixd['dwell_time_in_seconds'].resample(freq).count()
        ws[k] = ws[k][ws[k].index.isin(datetime_index)]

        ks.append(k)

  ts = pd.DataFrame(data = tss).fillna(method='pad').fillna(0) # Link Travel Time Time Series
  df_removed_mean = pd.DataFrame(data = removed_mean, index = ts.index).fillna(method='pad').fillna(method='bfill') # Removed Mean from Link Travel Time
  df_removed_scale = pd.DataFrame(data = removed_scale, index = ts.index).fillna(method='pad').fillna(method='bfill')
  w = pd.DataFrame(data = ws).fillna(0) # Link Travel Time Weights, e.g. number of measurements

  return (ts.index, ts.values, df_removed_mean.values, df_removed_scale.values, w.values, ks)

In [22]:

def roll(ix, ts, removed_mean, removed_scale, w, lags, preds):
    X = np.stack([np.roll(ts, i, axis = 0) for i in range(lags, 0, -1)], axis = 1)[lags:-preds,]
    Y = np.stack([np.roll(ts, -i, axis = 0) for i in range(0, preds, 1)], axis = 1)[lags:-preds,]
    Y_ix = ix[lags:-preds]
    Y_mean = np.stack([np.roll(removed_mean, -i, axis = 0) for i in range(0, preds, 1)], axis = 1)[lags:-preds,]
    Y_scale = np.stack([np.roll(removed_scale, -i, axis = 0) for i in range(0, preds, 1)], axis = 1)[lags:-preds,]
    w_y = np.stack([np.roll(w, -i, axis = 0) for i in range(0, preds, 1)], axis = 1)[lags:-preds,]

    return X, Y, Y_ix, Y_mean, Y_scale, w_y

In [23]:

"""Model Definition"""

def build_model(input_timesteps, output_timesteps, num_links):
    model = Sequential()
    model.add(BatchNormalization(name = 'batch_norm_0', input_shape = (input_timesteps, num_links, 1, 1)))
    model.add(ConvLSTM2D(name ='conv_lstm_1',
                         filters = 64, kernel_size = (10, 1),                       
                         padding = 'same', 
                         return_sequences = True))
    
    model.add(Dropout(0.2, name = 'dropout_1'))
    model.add(BatchNormalization(name = 'batch_norm_1'))

    model.add(ConvLSTM2D(name ='conv_lstm_2',
                         filters = 64, kernel_size = (5, 1), 
                         padding='same',
                         return_sequences = False))
    
    model.add(Dropout(0.1, name = 'dropout_2'))
    model.add(BatchNormalization(name = 'batch_norm_2'))
    
    model.add(Flatten())
    model.add(RepeatVector(output_timesteps))
    model.add(Reshape((output_timesteps, num_links, 1, 64)))
    
    model.add(ConvLSTM2D(name ='conv_lstm_3',
                         filters = 64, kernel_size = (10, 1), 
                         padding='same',
                         return_sequences = True))
    
    model.add(Dropout(0.1, name = 'dropout_3'))
    model.add(BatchNormalization(name = 'batch_norm_3'))
    
    model.add(ConvLSTM2D(name ='conv_lstm_4',
                         filters = 64, kernel_size = (5, 1), 
                         padding='same',
                         return_sequences = True))
    
    model.add(TimeDistributed(Dense(units=1, name = 'dense_1', activation = 'relu')))
    #model.add(TimeDistributed(Dense(units=1, name = 'dense_1', activation = 'relu')))
    #model.add(Dense(units=1, name = 'dense_2'))

    optimizer = RMSprop(lr=0.0001, rho=0.9, epsilon=1e-08, decay=0.9)
    model.compile(loss = "mse", optimizer = optimizer)
    return model


In [24]:

def info(msg):
    print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S') + " " + msg)

In [25]:

"""Load Data"""



lags = 4 * 13
preds = 3

n = len(data)

In [26]:

"""Train and Test"""

data_train = data[data['week_no'] < 36]
data_test = data[36 <= data['week_no']]
n_train = len(data_train)
n_test = len(data_test)
info('- Train size : {:>8} ({:.2f}%) '.format(n_train, 100. * n_train / n))
info('- Test size  : {:>8} ({:.2f}%) '.format(n_test, 100. * n_test / n))

(means, scales, low, upr) = fit_scale(data_train)

data_train_no , no_removed =  remove_outliers(data_train,low,upr)

# no_removed

data_test_no , no_removed =  remove_outliers(data_test,low,upr)
# no_removed

#data_train_no = data_train
#data_test_no = data_test

ix_train, ts_train, rm_mean_train, rm_scale_train, w_train, lns_train = transform(data_train_no, means, scales,datetime_index)
ix_test, ts_test, rm_mean_test, rm_scale_test, w_test, lns_test = transform(data_test_no, means, scales,datetime_index)

#Create rolling window tensor
X_train, Y_train, Y_ix_train, Y_rm_mean_train, Y_scale_train, Y_w_train = roll(ix_train, ts_train, rm_mean_train, rm_scale_train, w_train, lags, preds)
X_test, Y_test, Y_ix_test, Y_rm_mean_test, Y_scale_test, Y_w_test = roll(ix_test, ts_test, rm_mean_test, rm_scale_test, w_test, lags, preds)

X_train = X_train[:,:,:,np.newaxis,np.newaxis]
Y_train = Y_train[:,:,:,np.newaxis,np.newaxis]
X_test = X_test[:,:,:,np.newaxis,np.newaxis]
Y_test = Y_test[:,:,:,np.newaxis,np.newaxis]

# info('- X_train shape : {:>20}    X_test shape : {:>20}'.format(X_train.shape, X_test.shape))
# info('- Y_train shape : {:>20}    Y_test shape : {:>20}'.format(Y_train.shape, Y_test.shape))

2023-09-20 15:00:07 - Train size :    74041 (80.76%) 
2023-09-20 15:00:07 - Test size  :    17642 (19.24%) 


<ipython-input-17-0b257dfc7884>:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  datetime_series = pd.Series()
<ipython-input-17-0b257dfc7884>:9: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  daily_time_range = pd.Series(index = pd.date_range(begin_timestamp,end_timestamp,freq = freq,closed='left'))
<ipython-input-17-0b257dfc7884>:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  daily_time_range = pd.Series(index = pd.date_range(begin_timestamp,end_timestamp,freq = freq,closed='left'))
<ipython-input-17-0b257dfc7884>:9: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  daily_time_range = pd.Series(index = pd.date_range(begin_timestamp,end_timestamp,freq = freq,closed='left'))
<ipython-input-17-0b257dfc788

TypeError: TypeError: unsupported format string passed to tuple.__format__

In [27]:

global_start_time = time.time()
early_stopping = EarlyStopping(monitor='val_loss', patience = 3)

model = build_model(lags, preds, len(lns_train))

# Train
history = model.fit(X_train, Y_train,
                    batch_size = 32, epochs = 1,
                    shuffle = False, validation_data = (X_test, Y_test),
                    verbose = 2, callbacks = [early_stopping])
model.save('models/ConvLSTM_3x15min_10x64-5x64-10x64-5x64_Comparison.h5') 

374/374 - 1442s - loss: 0.7285 - val_loss: 0.7038 - 1442s/epoch - 4s/step


/opt/python/envs/default/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [28]:

Y_true = Y_test.squeeze() * Y_scale_test + Y_rm_mean_test
Y_naive = Y_rm_mean_test
Y_pred = model.predict(X_test).squeeze() * Y_scale_test + Y_rm_mean_test

Y_true_total = np.sum(Y_true * Y_w_test, axis = 2).squeeze()
Y_naive_total = np.sum(Y_naive * Y_w_test, axis = 2).squeeze()
Y_pred_total = np.sum(Y_pred * Y_w_test, axis = 2).squeeze()
  

70/70 [==============================] - 27s 372ms/step


In [29]:
  
for t in range(preds):
    mask = Y_true_total[:,t] > 0
    Y_true_total_t = Y_true_total[mask, t] / 60
    Y_naive_total_t = Y_naive_total[mask, t] / 60
    Y_pred_total_t = Y_pred_total[mask, t] / 60  

    error_naive_total_t = (Y_naive_total_t - Y_true_total_t)
    error_lstm_total_t = (Y_pred_total_t - Y_true_total_t)

    mae_lstm = np.mean(np.abs(error_lstm_total_t))
    rmse_lstm = np.sqrt(np.mean((error_lstm_total_t)**2))
    mape_lstm = np.mean(np.abs(error_lstm_total_t) / Y_true_total_t) * 100

    info("- t + %d - ConvLSTM - MAE: %5.2f - RMSE: %5.2f - MAPE: %5.2f" % (t + 1, mae_lstm, rmse_lstm, mape_lstm))

model.summary()

2023-09-20 15:26:28 - t + 1 - ConvLSTM - MAE:  1.96 - RMSE:  2.57 - MAPE: 418032143629803.69
2023-09-20 15:26:28 - t + 2 - ConvLSTM - MAE:  2.01 - RMSE:  2.63 - MAPE: 418032143629807.00
2023-09-20 15:26:28 - t + 3 - ConvLSTM - MAE:  2.07 - RMSE:  2.69 - MAPE: 418032143629810.12
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_norm_0 (BatchNormaliz  (None, 52, 14, 1, 1)     4         
 ation)                                                          
                                                                 
 conv_lstm_1 (ConvLSTM2D)    (None, 52, 14, 1, 64)     166656    
                                                                 
 dropout_1 (Dropout)         (None, 52, 14, 1, 64)     0         
                                                                 
 batch_norm_1 (BatchNormaliz  (None, 52, 14, 1, 64)    256       
 ation)                                  

In [1]:

output = pd.DataFrame(data = Y_pred[:,0,:], index = Y_ix_test, columns = lns_test)

output['Datetime']=output.index

output


NameError: NameError: name 'pd' is not defined

In [ ]:
pred_dwell_convolstm = output

In [ ]:
def download_csv(data,filename):
  filename= filename + '.csv'
  data.to_csv(filename, encoding = 'utf-8-sig',index= False)
  
download_csv(pred_dwell_convolstm,'/predicted_values/predicted_dwell_times_convolstm')


In [ ]:

# """Out of Fold Training"""



# global_start_time = time.time()
# early_stopping = EarlyStopping(monitor='val_loss', patience = 3)

# bootstrap_size_pct = 0.80
# test_window_pct = 0.05
# max_iter = 4

# lags = 4 * 8
# preds = 3

# hist = []
# for i in range(max_iter):

#     info("Current window: " + str(i))
    
#     # Devide into test and train
#     data_train = data[:int((bootstrap_size_pct + i * test_window_pct) * n)]
#     data_test = data[int((bootstrap_size_pct + i * test_window_pct) * n):int((bootstrap_size_pct + (i + 1) * test_window_pct) * n)]
#     n_train = len(data_train)
#     n_test = len(data_test)
#     info('- Train size : {:>8} ({:.2f}%) '.format(n_train, 100. * n_train / n))
#     info('- Test size  : {:>8} ({:.2f}%) '.format(n_test, 100. * n_test / n))
    
#     # Mean center and scale
#     (means, scales, low, upr) = fit_scale(data_train)
#     assert means.shape == (4 * 24 * 7, 32)
#     assert len(scales) == 32
#     assert low.shape == (4 * 24 * 7, 32)
#     assert upr.shape == (4 * 24 * 7, 32)
    
#     data_train_no = data_train
#     data_test_no = data_test
#     #data_train_no, n_outliers = remove_outliers(data_train, low, upr)
#     #info('- Removed {0} outliers ({1:.2f}%) from train'.format(n_outliers, 100.0 * n_outliers / len(data_train)))
#     #data_test_no, n_outliers = remove_outliers(data_test, low, upr)
#     #info('- Removed {0} outliers ({1:.2f}%) from test'.format(n_outliers, 100.0 * n_outliers / len(data_test)))
    
#     ix_train, ts_train, rm_mean_train, rm_scale_train, w_train, lns_train = transform(data_train_no, means, scales)
#     ix_test, ts_test, rm_mean_test, rm_scale_test, w_test, lns_test = transform(data_test_no, means, scales)

# X_train, Y_train, Y_ix_train, Y_rm_mean_train, Y_scale_train, Y_w_train = roll(ix_train, ts_train, rm_mean_train, rm_scale_train, w_train, lags, preds)
# X_test, Y_test, Y_ix_test, Y_rm_mean_test, Y_scale_test, Y_w_test = roll(ix_test, ts_test, rm_mean_test, rm_scale_test, w_test, lags, preds)
# X_train = X_train[:,:,:,np.newaxis,np.newaxis]
# Y_train = Y_train[:,:,:,np.newaxis,np.newaxis]
# X_test = X_test[:,:,:,np.newaxis,np.newaxis]
# Y_test = Y_test[:,:,:,np.newaxis,np.newaxis]
    
# info('- X_train shape : {:>20}    X_test shape : {:>20}'.format(X_train.shape, X_test.shape))
# info('- Y_train shape : {:>20}    Y_test shape : {:>20}'.format(Y_train.shape, Y_test.shape))
    
# model = build_model(lags, preds, len(lns_train))
    
#     # Train
# history = model.fit(X_train, Y_train,
#                         batch_size = 128, epochs = 30,
#                         shuffle = False, validation_data = (X_test, Y_test),
#                         verbose = 2) #, callbacks = [csv_logger, early_stopping])
# hist.append(history)
# model.save('models/ConvLSTM_3x15min_10x64-5x64-10x64-5x64_' + str(i) + '.h5') 

# Y_true = Y_test.squeeze() * Y_scale_test + Y_rm_mean_test
# Y_naive = Y_rm_mean_test
# Y_pred = model.predict(X_test).squeeze() * Y_scale_test + Y_rm_mean_test
        
# Y_true_total = np.sum(Y_true * Y_w_test, axis = 2).squeeze()
# Y_naive_total = np.sum(Y_naive * Y_w_test, axis = 2).squeeze()
# Y_pred_total = np.sum(Y_pred * Y_w_test, axis = 2).squeeze()


# multimodel

In [1]:
####### mme4bat / multimodel.py file #######

In [2]:
import sys
sys.path.append('/data/notebook_files/MME4BAT')

In [3]:
# !git clone https://github.com/HMMBaqir/DataOut

In [4]:
# !git clone https://github.com/HMMBaqir/gps2gtfs

In [5]:
# !git clone https://github.com/aaivu/MME4BAT


In [6]:
# -*- coding: utf-8 -*-
"""multimodel runtime.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1HMHvnKxsSdNt23LoSvqo-lvj5WwyslK_
"""

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score





In [ ]:

path= '/data/notebook_files/DataOut/processed_data/bus_running_times_feature_added_all.csv'
run_times = pd.read_csv(path)

path= '/data/notebook_files/DataOut/processed_data/bus_stop_times_feature_added_all.csv'
dwell_times = pd.read_csv(path)

KeyboardInterrupt: KeyboardInterrupt: interrupted by user

In [ ]:

df = dwell_times

df = df.loc[(df['time_of_day']>5.45) & (df['time_of_day']<19)]

df1 = df[df['direction']==1]

test1 = df1[df1['week_no']>16]
train1 = df1[df1['week_no']<17]

X = df1[['deviceid','bus_stop','day_of_week','weekday/end','time_of_day','dt(t-1)','dt(t-2)','dt(w-1)','dt(w-2)','dt(w-3)','dt(n-1)','dt(n-2)','dt(n-3)','temp', 'precip', 'holiday']]
y = df1[['dwell_time_in_seconds']]

In [ ]:

import xgboost as xgb
xgb=  xgb.XGBRegressor(colsample_bytree = 0.3, learning_rate = 0.1,max_depth = 5, alpha = 10, n_estimators = 1000)

Xtrain = train1[['deviceid','bus_stop','day_of_week','weekday/end','time_of_day','dt(t-1)','dt(t-2)','dt(w-1)','dt(w-2)','dt(w-3)','dt(n-1)','dt(n-2)','dt(n-3)','temp', 'precip', 'holiday']]
ytrain = train1[['dwell_time_in_seconds']]

Xtest = test1[['deviceid','bus_stop','day_of_week','weekday/end','time_of_day','dt(t-1)','dt(t-2)','dt(w-1)','dt(w-2)','dt(w-3)','dt(n-1)','dt(n-2)','dt(n-3)','temp', 'precip', 'holiday']]
ytest = test1[['dwell_time_in_seconds']]

xgb.fit(Xtrain,ytrain)
pred_xg = xgb.predict(Xtest) 
rmse = np.sqrt(mean_squared_error(ytest, pred_xg)) 

print("RMSE (1): %f" % (rmse))

mape = mean_absolute_percentage_error(ytest, pred_xg)
print("MAPE (1): %f" % (mape)) 

mae = mean_absolute_error(ytest, pred_xg)
print("MAE (1): %f" % (mae)) 


r2 = r2_score(ytest, pred_xg)
print("r2 (1): %f" % (r2))

pred_xg

In [ ]:
########### prediction added to test1 dataframe
test1.reset_index(drop = True, inplace = True)

pred_xg = pd.Series(pred_xg, name='XGBoost')


pred =test1.merge(pred_xg,left_index=True, right_index=True)


pred_dwell = pred

pred_dwell

In [ ]:

###########################################################running time model#################
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

df = run_times

df = df.loc[(df['time_of_day']>=6) & (df['time_of_day']<19)]

df1 = df[df['direction']==1]

test1 = df1[df1['week_no']>=36]
train1 = df1[df1['week_no']<36]

X = df[['deviceid','segment','day_of_week','length','weekday/end','time_of_day','rt(t-1)','rt(t-2)','rt(w-1)','rt(w-2)','rt(w-3)','rt(n-1)','rt(n-2)','rt(n-3)','temp', 'precip', 'holiday']]

y = df[['run_time_in_seconds']]



import xgboost as xgb
xgb=  xgb.XGBRegressor(colsample_bytree = 0.3, learning_rate = 0.1,max_depth = 5, alpha = 10, n_estimators = 1000)

Xtrain = train1[['deviceid','segment','day_of_week','length', 'Sunday/holiday', 'saturday','time_of_day','rt(t-1)','rt(t-2)','rt(w-1)','rt(w-2)','rt(w-3)','rt(n-1)','rt(n-2)','rt(n-3)','temp', 'precip']]
ytrain = train1[['run_time_in_seconds']]

Xtest = test1[['deviceid','segment','day_of_week','length', 'Sunday/holiday', 'saturday','time_of_day','rt(t-1)','rt(t-2)','rt(w-1)','rt(w-2)','rt(w-3)','rt(n-1)','rt(n-2)','rt(n-3)','temp', 'precip']]
ytest = test1[['run_time_in_seconds']]

xgb.fit(Xtrain,ytrain)
pred_xg = xgb.predict(Xtest) 
rmse = np.sqrt(mean_squared_error(ytest, pred_xg)) 

print("RMSE (1): %f" % (rmse))

mape = mean_absolute_percentage_error(ytest, pred_xg)
print("MAPE (1): %f" % (mape)) 

mae = mean_absolute_error(ytest, pred_xg)
print("MAE (1): %f" % (mae)) 


r2 = r2_score(ytest, pred_xg)
print("r2 (1): %f" % (r2))




In [ ]:
test1.reset_index(drop = True, inplace = True)

pred_xg = pd.Series(pred_xg, name='XGBoost')


pred =test1.merge(pred_xg,left_index=True, right_index=True)


pred_run = pred

pred_run

In [ ]:


def download_csv(data,filename):
  filename= filename + '.csv'
  data.to_csv(filename, encoding = 'utf-8-sig',index= False)
  
download_csv(pred_run,'/predicted_values/predicted_running_times_xg_boost')

download_csv(pred_dwell,'/predicted_values/predicted_dwell_times_xg_boost')

In [ ]:
##########pivoting so we get one row per trip with run time , and dwell time values########

run_true = pred_run.pivot_table(index='trip_id', columns='segment', values='run_time_in_seconds')
run_true = run_true.fillna(50)

run_pred = pred_run.pivot_table(index='trip_id', columns='segment', values='XGBoost')
run_pred = run_pred.fillna(50)

dwell_true = pred_dwell.pivot_table(index='trip_id', columns='bus_stop', values='dwell_time_in_seconds')
dwell_true = dwell_true.fillna(0)

dwell_pred = pred_dwell.pivot_table(index='trip_id', columns='bus_stop', values='XGBoost')
dwell_pred = dwell_pred.fillna(0)






In [ ]:

#merge and get sum of dwell , running time for total sum
true =run_true.merge(dwell_true,left_index=True, right_index=True)

pred =run_pred.merge(dwell_pred,left_index=True, right_index=True)

pred['total_pred'] = pred[list(pred.columns)].sum(axis=1)

true['total_pred'] = true[list(true.columns)].sum(axis=1)

In [ ]:
# get errors 
rmse = np.sqrt(mean_squared_error(true['total_pred'], pred['total_pred'])) 

print("RMSE (1): %f" % (rmse))

mape = mean_absolute_percentage_error(true['total_pred'],pred['total_pred'])
print("MAPE (1): %f" % (mape)) 

mae = mean_absolute_error(true['total_pred'], pred['total_pred'])
print("MAE (1): %f" % (mae)) 


r2 = r2_score(true['total_pred'], pred['total_pred'])
print("r2 (1): %f" % (r2))


In [ ]:

# ################################lining up pred dwell and pred run so we can add them (by dividing into total run, dwell for every 15 min )
# pred_dwell['DateTime'] = pd.to_datetime(pred_dwell['date'] + ' ' + pred_dwell['arrival_time'])
# ref_freq = '15min'
# ix = pd.DatetimeIndex(pd.to_datetime(pred_dwell['DateTime'])).floor(ref_freq)
# pred_dwell["DateTimeRef"] = ix

# pred_run['DateTime'] = pd.to_datetime(pred_run['date'] + ' ' + pred_run['start_time'])
# ref_freq = '15min'
# ix = pd.DatetimeIndex(pd.to_datetime(pred_run['DateTime'])).floor(ref_freq)
# pred_run["DateTimeRef"] = ix

# pred_run
# ############################### lined up ####################################3

In [ ]:
# #import convlstm predictions
# path_convlstm= '/Predicted values/convlstm.csv'

# convlstm = pd.read_csv(path_convlstm)

In [ ]:


# convlstm = convlstm.set_index('Datetime')

# convlstm.index = pd.to_datetime(convlstm.index)

In [ ]:

# date_list = convlstm.index.tolist()
# pred_run= pred_run[pred_run['DateTimeRef'].isin(date_list)]


In [ ]:

# pred_run['covlstm'] =''

# for segment, group in pred_run.groupby('segment'):
#   for row_index, row in group.iterrows():
#     date_time = row['DateTimeRef'] 
#     pred_run.at[row_index,'covlstm'] = convlstm.loc[date_time,str(segment)]

# pred_run

In [ ]:

# download_csv(pred_run,'predicted_running_times_combined')

In [ ]:


# print('results for convlstm:')
# rmse = np.sqrt(mean_squared_error(pred_run['run_time_in_seconds'], pred_run['covlstm'])) 

# print("RMSE (1): %f" % (rmse))

# mape = mean_absolute_percentage_error(pred_run['run_time_in_seconds'],pred_run['covlstm'])
# print("MAPE (1): %f" % (mape)) 

# mae = mean_absolute_error(pred_run['run_time_in_seconds'], pred_run['covlstm'])
# print("MAE (1): %f" % (mae)) 


# r2 = r2_score(pred_run['run_time_in_seconds'], pred_run['covlstm'])
# print("r2 (1): %f" % (r2))


# print('results for XGBoost:')
# rmse = np.sqrt(mean_squared_error(pred_run['run_time_in_seconds'], pred_run['XGBoost'])) 

# print("RMSE (1): %f" % (rmse))

# mape = mean_absolute_percentage_error(pred_run['run_time_in_seconds'],pred_run['XGBoost'])
# print("MAPE (1): %f" % (mape)) 

# mae = mean_absolute_error(pred_run['run_time_in_seconds'], pred_run['XGBoost'])
# print("MAE (1): %f" % (mae)) 


# r2 = r2_score(pred_run['run_time_in_seconds'], pred_run['XGBoost'])
# print("r2 (1): %f" % (r2))

In [ ]:

# run_true = pred_run_f.pivot_table(index='trip_id', columns='segment', values='run_time_in_seconds')
# run_true = run_true.fillna(50)

# run_pred = pred_run_f.pivot_table(index='trip_id', columns='segment', values='cnn_lstm')
# run_pred = run_pred.fillna(50)

# dwell_true = pred_dwell_f.pivot_table(index='trip_id', columns='bus_stop', values='dwell_time_in_seconds')
# dwell_true = dwell_true.fillna(0)

# dwell_pred = pred_dwell_f.pivot_table(index='trip_id', columns='bus_stop', values='cnn_lstm')
# dwell_pred = dwell_pred.fillna(0)

# true =run_true.merge(dwell_true,left_index=True, right_index=True)
# pred =run_pred.merge(dwell_pred,left_index=True, right_index=True)

# pred['total_pred'] = pred[list(pred.columns)].sum(axis=1)
# true['total_true'] = true[list(true.columns)].sum(axis=1)

# rmse = np.sqrt(mean_squared_error(true['total_true'], pred['total_pred'])) 

# print("RMSE (1): %f" % (rmse))

# mape = mean_absolute_percentage_error(true['total_true'],pred['total_pred'])
# print("MAPE (1): %f" % (mape)) 

# mae = mean_absolute_error(true['total_true'], pred['total_pred'])
# print("MAE (1): %f" % (mae)) 


# r2 = r2_score(true['total_true'], pred['total_pred'])
# print("r2 (1): %f" % (r2))

In [ ]:

# download_csv(pred_run_f,'predicted_running_times')

# download_csv(pred_dwell_f,'predicted_dwell_times')

In [ ]:

# def correlation_heatmap(train):
#     correlations = train.corr()

#     fig, ax = plt.subplots(figsize=(10,10))
#     sns.heatmap(correlations, vmax=1.0, center=0, fmt='.2f', cmap="YlGnBu",
#                 square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .70}
#                 )
#     plt.show();
    
# correlation_heatmap( run_times[['deviceid','segment','day_of_week','length','weekday/end','time_of_day','rt(t-1)','rt(t-2)','rt(w-1)','rt(w-2)','rt(w-3)','rt(n-1)','rt(n-2)','rt(n-3)','temp', 'precip', 'holiday','run_time_in_seconds']])

# correlation_heatmap(dwell_times[['deviceid','bus_stop','day_of_week','weekday/end','time_of_day','dt(t-1)','dt(t-2)','dt(w-1)','dt(w-2)','dt(w-3)','dt(n-1)','dt(n-2)','dt(n-3)','temp', 'precip', 'holiday','dwell_time_in_seconds']])

In [ ]:
# from gps2gtfs.pipeline.trip import run


# if __name__ == "__main__":
#     raw_gps_data_path = "/data/notebook_files/gps2gtfs/DSE_raw_data/raw_data/digana_2022_07.csv"
#     trip_terminals_data_path = "/data/notebook_files/gps2gtfs/DSE_raw_data/raw_data/bus_terminals_654.csv"
#     terminals_buffer_radius = 100

#     run(
        
#         raw_gps_data_path,
#         trip_terminals_data_path,
#         terminals_buffer_radius,
#     )

In [ ]:
# # modify code below using code given above  
# from gps2gtfs.pipeline.trip_stop import run

# if __name__ == "__main__":
#     raw_gps_data_path = "/data/notebook_files/gps2gtfs/DSE_raw_data/raw_data/digana_2022_07.csv"
#     trip_terminals_data_path = "/data/notebook_files/gps2gtfs/DSE_raw_data/raw_data/bus_terminals_654.csv"
#     stops_data_path = "/data/notebook_files/gps2gtfs/DSE_raw_data/raw_data/bus_stops_654.csv"
#     terminals_buffer_radius = 100
#     stops_buffer_radius = 50
#     stops_extended_buffer_radius = 100

#     run(
#         raw_gps_data_path,
#         trip_terminals_data_path,
#         stops_data_path,
#         terminals_buffer_radius,
#         stops_buffer_radius,
#         stops_extended_buffer_radius,
#     )

2023-09-14 15:17:08 - gps2gtfs - INFO - trip_stop.py:run - Pipeline method called !
2023-09-14 15:17:08 - gps2gtfs - INFO - trip_stop.py:run - Starting Pipeline for extracting Trip Data
2023-09-14 15:17:09 - gps2gtfs - INFO - data_io_converter.py:read_csv_file - Successfully read the Raw GPS data file in the path /data/notebook_files/gps2gtfs/DSE_raw_data/raw_data/digana_2022_07.csv
2023-09-14 15:17:09 - gps2gtfs - INFO - data_io_converter.py:read_csv_file - Successfully read the Trip terminals data file in the path /data/notebook_files/gps2gtfs/DSE_raw_data/raw_data/bus_terminals_654.csv
2023-09-14 15:17:09 - gps2gtfs - INFO - data_io_converter.py:read_csv_file - Successfully read the Stops data file in the path /data/notebook_files/gps2gtfs/DSE_raw_data/raw_data/bus_stops_654.csv
2023-09-14 15:17:09 - gps2gtfs - INFO - load_from_csv.py:load_data_for_trip_stop_pipeline - Data Loaded successfully for pipeline
2023-09-14 15:17:09 - gps2gtfs - INFO - trip_stop.py:run - Successfully read 

# dwell time classifier

In [ ]:
#####    dwelltime_classification #############

In [ ]:
# -*- coding: utf-8 -*-
"""dwelltime classification.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1F2Z5ixi2Du97jEHqKepPmJ_Sy8lMVFck
"""

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime,date

import xgboost as xgb
from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

from yellowbrick.regressor import residuals_plot
from yellowbrick.regressor import prediction_error

from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV


In [ ]:


path= '/data/notebook_files/DataOut/processed_data/bus_stop_times_feature_added_all.csv'

df = pd.read_csv(path)

In [ ]:

def condition(x):
  if x == 0:
    return 0
  else:
    return 1

df['dwell/pass']= df['dwell_time_in_seconds'].apply(condition)

df

In [ ]:

test = df[df['week_no']>=36]
train = df[df['week_no']<36]

test.reset_index(drop = True, inplace = True)

In [ ]:
###############   xgb classifier to find if dwell time is positive
import xgboost as xgb
xgb=  xgb.XGBClassifier()

Xtrain = train[['deviceid','bus_stop','day_of_week', 'Sunday/holiday', 'saturday','time_of_day','dt(w-1)', 'dt(w-2)', 'dt(w-3)', 'dt(t-1)','dt(t-2)', 'dt(n-1)', 'dt(n-2)', 'dt(n-3)','temp', 'precip','rt(n-1)']]
ytrain = train[['dwell/pass']]

Xtest = test[['deviceid','bus_stop','day_of_week', 'Sunday/holiday', 'saturday','time_of_day','dt(w-1)', 'dt(w-2)', 'dt(w-3)', 'dt(t-1)','dt(t-2)', 'dt(n-1)', 'dt(n-2)', 'dt(n-3)','temp', 'precip','rt(n-1)']]
ytest = test[['dwell/pass']]

xgb.fit(Xtrain,ytrain)
pred_xg = xgb.predict(Xtest) 

accuracy_score(ytest, pred_xg)

confusion_matrix(ytest, pred_xg)

pred_xgt = xgb.predict(Xtrain) 

accuracy_score(ytrain, pred_xgt)

pred_xg = pd.Series(pred_xg, name='XGBoost_class')
pred =test.merge(pred_xg,left_index=True, right_index=True)

pred['XGBoost_class'].value_counts()

train = train[train['dwell/pass']!=0]
test_r = pred[pred['XGBoost_class']!=0]

test_r

In [ ]:

import xgboost as xgb
xgb=  xgb.XGBRegressor(colsample_bytree = 0.7, learning_rate = 0.1,max_depth = 6, alpha = 10, n_estimators = 1000)

Xtrain = train[['deviceid','bus_stop','day_of_week', 'Sunday/holiday', 'saturday','time_of_day','dt(w-1)', 'dt(w-2)', 'dt(w-3)', 'dt(t-1)','dt(t-2)', 'dt(n-1)', 'dt(n-2)', 'dt(n-3)','temp', 'precip','rt(n-1)']]
ytrain = train[['dwell_time_in_seconds']]

Xtest = test_r[['deviceid','bus_stop','day_of_week', 'Sunday/holiday', 'saturday','time_of_day','dt(w-1)', 'dt(w-2)', 'dt(w-3)', 'dt(t-1)','dt(t-2)', 'dt(n-1)', 'dt(n-2)', 'dt(n-3)','temp', 'precip','rt(n-1)']]
ytest = test_r[['dwell_time_in_seconds']]

xgb.fit(Xtrain,ytrain)
pred_xg_r = xgb.predict(Xtest) 
rmse = np.sqrt(mean_squared_error(ytest, pred_xg_r)) 

print("RMSE (1): %f" % (rmse))

mape = mean_absolute_percentage_error(ytest, pred_xg_r)
print("MAPE (1): %f" % (mape)) 

mae = mean_absolute_error(ytest, pred_xg_r)
print("MAE (1): %f" % (mae)) 


r2 = r2_score(ytest, pred_xg_r)
print("r2 (1): %f" % (r2))

In [ ]:

test_r.reset_index(drop = True, inplace = True)

pred_xg_r = pd.Series(pred_xg_r, name='XGBoost_reg')
pred_r =test_r.merge(pred_xg_r,left_index=True, right_index=True)
pred_r

In [ ]:

pred_r.drop(['XGBoost_class'], axis=1,inplace=True)

pred_r.rename(columns = {'XGBoost_reg':'XGBoost_class'}, inplace = True)


pred_c = pred[pred['XGBoost_class']==0]

pred_dwell = pd.concat([pred_c, pred_r])

pred_dwell = pred_dwell.sort_values(['trip_id', 'bus_stop'])

pred_dwell.reset_index(drop = True, inplace = True)

pred_dwell

In [ ]:

rmse = np.sqrt(mean_squared_error(pred_dwell['dwell_time_in_seconds'], pred_dwell['XGBoost_class'])) 

print("RMSE (1): %f" % (rmse))

mape = mean_absolute_percentage_error(pred_dwell['dwell_time_in_seconds'], pred_dwell['XGBoost_class'])
print("MAPE (1): %f" % (mape)) 

mae = mean_absolute_error(pred_dwell['dwell_time_in_seconds'], pred_dwell['XGBoost_class'])
print("MAE (1): %f" % (mae)) 


r2 = r2_score(pred_dwell['dwell_time_in_seconds'], pred_dwell['XGBoost_class'])
print("r2 (1): %f" % (r2))

(pred_dwell['XGBoost_class'] < 0).sum()

pred = pred_dwell

pred

In [ ]:

pred['DateTime'] = pd.to_datetime(pred['date'] + ' ' + pred['arrival_time'])
ref_freq = '15min'
ix = pd.DatetimeIndex(pd.to_datetime(pred['DateTime'])).floor(ref_freq)
pred["DateTimeRef"] = ix

path= '/Predicted values/predicted_dwell_times_xg_boost.csv'

pred_dwell = pd.read_csv(path)

pred = pred[pred['DateTimeRef'].isin(pred_dwell['DateTimeRef'].tolist())]

pred = pred.sort_values(['trip_id', 'bus_stop'])

pred.reset_index(drop = True, inplace = True)

pred_dwell = pred_dwell.sort_values(['trip_id', 'bus_stop'])

pred_dwell.reset_index(drop = True, inplace = True)

pred_dwell =pred_dwell.merge(pred['XGBoost_class'],left_index=True, right_index=True)

def download_csv(data,filename):
  filename= filename + '.csv'
  data.to_csv(filename, encoding = 'utf-8-sig',index= False)
  files.download(filename)

download_csv(pred_dwell,'predicted_dwell_times')

ytest

# Ensemble models